In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report,accuracy_score,ConfusionMatrixDisplay,confusion_matrix,precision_score,recall_score,roc_curve,roc_auc_score,balanced_accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import datetime
from dataclasses import dataclass
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from scipy.ndimage import gaussian_filter1d
from itertools import combinations as comb
from pyts.image import RecurrencePlot
import PIL
from sklearn.utils import class_weight
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,StratifiedKFold
from unidecode import unidecode

In [25]:
train_data = pd.read_csv('./dataset/dataset_1/train_motion_data.csv')
test_data = pd.read_csv('./dataset/dataset_1/test_motion_data.csv')
print(train_data.shape)
print(test_data.shape)

(3644, 8)
(3084, 8)


In [26]:
train_data.head()

,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,Class,Timestamp
0,0.000000,0.000000,0.000000,0.059407,-0.174707,0.101938,NORMAL,3581629
1,-1.624864,-1.082492,-0.204183,-0.028558,0.051313,0.135536,NORMAL,3581630
2,-0.594660,-0.122410,0.220502,-0.019395,-0.029322,0.087888,NORMAL,3581630
3,0.738478,-0.228456,0.667732,0.069791,-0.029932,0.054902,NORMAL,3581631
4,0.101741,0.777568,-0.066730,0.030696,-0.003665,0.054902,NORMAL,3581631


In [27]:
print(train_data['Class'].value_counts())

SLOW          1331
NORMAL        1200
AGGRESSIVE    1113
Name: Class, dtype: int64


In [28]:
print(test_data['Class'].value_counts())

SLOW          1273
NORMAL         997
AGGRESSIVE     814
Name: Class, dtype: int64


In [5]:
test_data.head()

,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,Class,Timestamp
0,0.758194,-0.217791,0.457263,0.000000,0.000000,0.000000,AGGRESSIVE,818922
1,0.667560,-0.038610,0.231416,-0.054367,-0.007712,0.225257,AGGRESSIVE,818923
2,2.724449,-7.584121,2.390926,0.023824,0.013668,-0.038026,AGGRESSIVE,818923
3,2.330950,-7.621754,2.529024,0.056810,-0.180587,-0.052076,AGGRESSIVE,818924
4,2.847215,-6.755621,2.224640,-0.031765,-0.035201,0.035277,AGGRESSIVE,818924


In [10]:
x_train = train_data[['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ']]
train_data.Class=train_data.Class.replace(['AGGRESSIVE', 'NORMAL', 'SLOW'],[1,2,3])
y_train = train_data[['Class']]

In [11]:
x_train

,AccX,AccY,AccZ,GyroX,GyroY,GyroZ
0,0.000000,0.000000,0.000000,0.059407,-0.174707,0.101938
1,-1.624864,-1.082492,-0.204183,-0.028558,0.051313,0.135536
2,-0.594660,-0.122410,0.220502,-0.019395,-0.029322,0.087888
3,0.738478,-0.228456,0.667732,0.069791,-0.029932,0.054902
4,0.101741,0.777568,-0.066730,0.030696,-0.003665,0.054902
...,...,...,...,...,...,...
3639,0.915688,-2.017489,1.687505,0.450360,0.384845,-1.236468
3640,-1.934203,0.914925,-0.096013,0.321468,0.649350,-0.477162
3641,-0.222845,0.747304,-0.887430,0.361174,-0.406836,0.054291
3642,-0.349423,0.067261,0.394368,-0.132405,0.020159,-0.004963


In [12]:
y_train

,Class
0,2
1,2
2,2
3,2
4,2
...,...
3639,3
3640,3
3641,3
3642,3


In [13]:
x_test = test_data[['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ']]
test_data.Class=test_data.Class.replace(['AGGRESSIVE', 'NORMAL', 'SLOW'],[1,2,3])
y_test = test_data[['Class']]

In [14]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)

c:\Users\MILAB\Anaconda3\envs\torchpy37\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier()

In [22]:
# rf.score(x_test, y_test)
y_pred = rf.predict(x_test)
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test, digits=4))

0.41245136186770426
              precision    recall  f1-score   support

           1     0.4165    0.4154    0.4160       816
           2     0.3149    0.3344    0.3244       939
           3     0.4863    0.4658    0.4758      1329

    accuracy                         0.4125      3084
   macro avg     0.4059    0.4052    0.4054      3084
weighted avg     0.4156    0.4125    0.4139      3084

